In [2]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [3]:
df_for = pd.read_csv('datos/api_foursquare.csv')
df_ae = pd.read_csv('datos/df_aemet_final.csv')
df_muni = pd.read_csv('datos/municipios.csv')

In [4]:
df_for.head(2)                   #actvidades y movidas

,Unnamed: 0,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773


In [5]:
df_ae.head(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001


In [6]:
df_muni.head(2)

,Unnamed: 0,acebeda-la,ajalvir,alameda-del-valle,alamo-el,alcala-de-henares,alcobendas,alcorcon,aldea-del-fresno,algete,...,villamanta,villamantilla,villanueva-de-la-canada,villanueva-del-pardillo,villanueva-de-perales,villar-del-olmo,villarejo-de-salvanes,villaviciosa-de-odon,villavieja-del-lozoya,zarzalejo
0,0,41.0869583,40.5342302,40.9177178,40.2307296,40.4818396,40.5400082,40.3492998,40.3215212,40.5961652,...,40.2990939,40.3421033,40.4472106,40.4949232,40.3466066,40.3364465,40.1683582,40.3573787,41.0063622,40.5483016
1,1,-3.624399,-3.4807818,-3.8438216,-3.9905893,-3.3644973,-3.6358494,-3.8284295,-4.19936,-3.4974246,...,-4.1090399,-4.1300303,-4.002795,-3.9600065,-4.1012616,-3.2355624,-3.2738764,-3.9002334,-3.6716291,-4.1815899


In [7]:
try:

    conexion1 = psycopg2.connect(database='Forq',
                                    user= 'postgres',
                                    password='admin',
                                    host= 'localhost',
                                    port='5432')
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('La contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('Error de conexion')
    else:
        print(f'ocurrio el error {e}')

cursor = conexion1.cursor()

In [8]:
query_creacion_municipios="""
CREATE TABLE IF NOT EXISTS localidades (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    municipios TEXT,
    latitud FLOAT,
    longitud FLOAT)"""


In [9]:
cursor.execute(query_creacion_municipios)
conexion1.commit()

In [10]:
df_muni = df_muni.T
df_muni.head(3)



,0,1,2
Unnamed: 0,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda
ajalvir,40.5342302,-3.4807818,Ajalvir


In [11]:
df_muni.drop(index='Unnamed: 0', inplace=True)


In [12]:
df_muni.head()

,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda
ajalvir,40.5342302,-3.4807818,Ajalvir
alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
alamo-el,40.2307296,-3.9905893,El Álamo
alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares


In [13]:
df_muni['id']= 0

In [14]:
df_muni.head(2)

,0,1,2,id
acebeda-la,41.0869583,-3.624399,La Acebeda,0
ajalvir,40.5342302,-3.4807818,Ajalvir,0


In [15]:
rango = range(1, 177)
df_muni['id']= rango

In [16]:
df_muni.head()

,0,1,2,id
acebeda-la,41.0869583,-3.624399,La Acebeda,1
ajalvir,40.5342302,-3.4807818,Ajalvir,2
alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle,3
alamo-el,40.2307296,-3.9905893,El Álamo,4
alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares,5


In [24]:
set(df_muni.index.unique()) - set(df_for["municipio"].unique())

{'acebeda-la',
 'alameda-del-valle',
 'aldea-del-fresno',
 'ambite',
 'anchuelo',
 'arroyomolinos',
 'atazar-el',
 'belmonte-de-tajo',
 'berrueco-el',
 'berzosa-del-lozoya',
 'brea-de-tajo',
 'cadalso-de-los-vidrios',
 'campo-real',
 'canencia',
 'carabana',
 'casarrubuelos',
 'cenicientos',
 'cervera-de-buitrago',
 'colmenar-del-arroyo',
 'corpa',
 'estremera',
 'fresnedillas-de-la-oliva',
 'fresno-de-torote',
 'fuentiduena-de-tajo',
 'garganta-de-los-montes',
 'gargantilla-del-lozoya-y-pinilla-de-buitrago',
 'hiruela-la',
 'horcajo-de-la-sierra-aoslos',
 'horcajuelo-de-la-sierra',
 'lozoya',
 'lozoyuela-navas-sieteiglesias',
 'madarcos',
 'molar-el',
 'montejo-de-la-sierra',
 'moraleja-de-enmedio',
 'morata-de-tajuna',
 'navalafuente',
 'navarredonda-y-san-mames',
 'nuevo-baztan',
 'olmeda-de-las-fuentes',
 'orusco-de-tajuna',
 'pedrezuela',
 'pelayos-de-la-presa',
 'perales-de-tajuna',
 'pezuela-de-las-torres',
 'pinilla-del-valle',
 'pinuecar-gandullas',
 'pozuelo-del-rey',
 'prade

In [21]:
df_for["municipio"].unique

array(['ajalvir', 'alamo-el', 'alcala-de-henares', 'alcobendas',
       'alcorcon', 'algete', 'alpedrete', 'aranjuez', 'arganda-del-rey',
       'batres', 'becerril-de-la-sierra', 'boadilla-del-monte',
       'boalo-el', 'braojos', 'brunete', 'buitrago-del-lozoya',
       'bustarviejo', 'cabanillas-de-la-sierra', 'camarma-de-esteruelas',
       'cercedilla', 'chapineria', 'chinchon', 'ciempozuelos', 'cobena',
       'collado-mediano', 'collado-villalba', 'colmenar-de-oreja',
       'colmenarejo', 'colmenar-viejo', 'coslada', 'cubas-de-la-sagra',
       'daganzo-de-arriba', 'escorial-el', 'fuenlabrada',
       'fuente-el-saz-de-jarama', 'galapagar', 'gascones', 'getafe',
       'grinon', 'guadalix-de-la-sierra', 'guadarrama',
       'hoyo-de-manzanares', 'humanes-de-madrid', 'loeches', 'madrid',
       'majadahonda', 'manzanares-el-real', 'meco', 'mejorada-del-campo',
       'miraflores-de-la-sierra', 'moralzarzal', 'mostoles',
       'navacerrada', 'navalagamella', 'navalcarnero', 'nav

In [18]:
df_muni.columns = ['latitud','longitud','municipio','id']

df_muni
df_muni['latitud'] = df_muni['latitud'].astype(float)
df_muni['longitud'] = df_muni['longitud'].astype(float)

In [19]:
lista_tuplas= [tuple(fila) for fila in df_muni.values]
tuple(df_muni.values[0])

(41.0869583, -3.624399, 'La Acebeda', 1)

In [20]:
for tupla in lista_tuplas:
    query_insercion_muni ="INSERT INTO localidades (latitud,longitud,municipios,id) VALUES (%s,%s,%s,%s)"
    valores=tupla

    cursor.execute(query_insercion_muni,valores)
    conexion1.commit()


UniqueViolation: llave duplicada viola restricción de unicidad «localidades_pkey»
DETAIL:  Ya existe la llave (id)=(1).
